# Load dataset and seperate into train/test/validation

In [3]:
import tensorflow_datasets as tfds
import tensorflow as tf

batch_size = 32
training, validation, test = tfds.load('cats_vs_dogs', split=['train[:70%]', 'train[70%:80%]', 'train[90%:]'], as_supervised=True, shuffle_files=True)

def preprocess(ds):
    ds = ds.map(lambda x, y: (tf.image.resize(x, (224, 224)), y))
    ds = ds.map(lambda x, y: (tf.keras.applications.vgg19.preprocess_input(x), y)) # converts to greyscale and zero-centers the data
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return ds

training = preprocess(training)
validation = preprocess(validation)
test = preprocess(test)

# Set up model

In [4]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Rescaling(1./255))
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss=tf.losses.BinaryCrossentropy(), metrics=['binary_accuracy'])

# Train model

I picked 4 pooling/conv layers because accuracy didn't seem improve much beyond that number of layers.

In [5]:
epochs = 5
model.fit(training, epochs=epochs, validation_data=validation)

Epoch 1/5
509/509 [==============================] - 106s 147ms/step - loss: 0.6011 - binary_accuracy: 0.6604 - val_loss: 0.5115 - val_binary_accuracy: 0.7538
Epoch 2/5
509/509 [==============================] - 74s 146ms/step - loss: 0.4540 - binary_accuracy: 0.7879 - val_loss: 0.4420 - val_binary_accuracy: 0.7877
Epoch 3/5
509/509 [==============================] - 76s 148ms/step - loss: 0.3764 - binary_accuracy: 0.8332 - val_loss: 0.4073 - val_binary_accuracy: 0.8126
Epoch 4/5
509/509 [==============================] - 75s 146ms/step - loss: 0.3056 - binary_accuracy: 0.8694 - val_loss: 0.4078 - val_binary_accuracy: 0.8311
Epoch 5/5
509/509 [==============================] - 75s 147ms/step - loss: 0.2305 - binary_accuracy: 0.9030 - val_loss: 0.4723 - val_binary_accuracy: 0.8285


# Evaluate on test set

In [6]:
print(model.metrics_names)
print(model.evaluate(test, verbose=0))

['loss', 'binary_accuracy']
[0.4418994188308716, 0.8271710872650146]


# Train VGG19

In [8]:
import numpy as np

model = tf.keras.Sequential()
model.add(tf.keras.applications.VGG19(include_top=False, pooling='avg'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.layers[0].trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

epochs = 5
model.fit(training, epochs=epochs, validation_data=validation)

Epoch 1/5
509/509 [==============================] - 171s 333ms/step - loss: 0.1461 - binary_accuracy: 0.9542 - val_loss: 0.0814 - val_binary_accuracy: 0.9759
Epoch 2/5
509/509 [==============================] - 170s 333ms/step - loss: 0.0539 - binary_accuracy: 0.9815 - val_loss: 0.0692 - val_binary_accuracy: 0.9772
Epoch 3/5
509/509 [==============================] - 170s 333ms/step - loss: 0.0413 - binary_accuracy: 0.9865 - val_loss: 0.0644 - val_binary_accuracy: 0.9802
Epoch 4/5
509/509 [==============================] - 170s 334ms/step - loss: 0.0347 - binary_accuracy: 0.9891 - val_loss: 0.0612 - val_binary_accuracy: 0.9820
Epoch 5/5
509/509 [==============================] - 170s 333ms/step - loss: 0.0304 - binary_accuracy: 0.9903 - val_loss: 0.0593 - val_binary_accuracy: 0.9837


# Evaluate VGG19 on test set

The VGG19 model performs much better than the regular CNN, likely due to the more robust model architecture and a larger training dataset.

In [9]:
print(model.metrics_names)
print(model.evaluate(test, verbose=0))

['loss', 'binary_accuracy']
[0.04546971619129181, 0.9888219833374023]


# Create and train new CNN with random transformations

In [12]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Rescaling(1./255))
model.add(tf.keras.layers.RandomFlip())
model.add(tf.keras.layers.RandomRotation((1, 1), fill_mode='nearest'))
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss=tf.losses.BinaryCrossentropy(), metrics=['binary_accuracy'])

epochs = 5
model.fit(training, epochs=epochs, validation_data=validation)

Epoch 1/5
509/509 [==============================] - 81s 157ms/step - loss: 0.6093 - binary_accuracy: 0.6545 - val_loss: 0.5696 - val_binary_accuracy: 0.7035
Epoch 2/5
509/509 [==============================] - 79s 156ms/step - loss: 0.5281 - binary_accuracy: 0.7369 - val_loss: 0.5056 - val_binary_accuracy: 0.7602
Epoch 3/5
509/509 [==============================] - 80s 156ms/step - loss: 0.4699 - binary_accuracy: 0.7771 - val_loss: 0.5191 - val_binary_accuracy: 0.7400
Epoch 4/5
509/509 [==============================] - 79s 154ms/step - loss: 0.4288 - binary_accuracy: 0.8028 - val_loss: 0.4163 - val_binary_accuracy: 0.8131
Epoch 5/5
509/509 [==============================] - 79s 155ms/step - loss: 0.3940 - binary_accuracy: 0.8232 - val_loss: 0.4113 - val_binary_accuracy: 0.8092


# Evaluate on test set

Performance is a little bit better, likely because the random transformations forced the model to learn fundemental patterns of the animals instead of learning the simple patterns present in the image.

In [13]:
print(model.metrics_names)
print(model.evaluate(test, verbose=0))

['loss', 'binary_accuracy']
[0.38249924778938293, 0.8207222819328308]
